In [107]:
from datasets import load_dataset, Dataset, DatasetDict
import re

# Filtering functions

In [7]:
def remove_duplicates(dataset, subset_columns: list):
    """ Remove duplicates based on columns subset"""
    # Convert to pandas DataFrame
    df = dataset.to_pandas()
    
    # Drop duplicates based on the specified columns
    deduplicated_df = df.drop_duplicates(subset=subset_columns, keep='first')
    
    # Convert back to Hugging Face dataset
    return Dataset.from_pandas(deduplicated_df)

def contains_code(text):
    """ Function to detect code in text """
    # Common code patterns: imports, function defs, loops, braces, semicolons, etc.
    code_patterns = [
        r'\bdef\b|\bclass\b|\bimport\b|\bfrom\b',       # Python
        r'\bfor\b|\bwhile\b|\bif\b|\belse\b',           # Control statements
        r'[{};<>]',                                     # Braces, semicolons (C/C++/Java/JS)
        r'#include\s*<.*?>',                            # C/C++ includes
        r'system\(.+\)|printf\(.+\)|cout\s*<<',         # C/C++ functions
        r'console\.log\(.+\)|function\s*\w*\s*\(',      # JS
        r'public|private|static|void|int|string',       # Java/C#
        r'\<\w+\>.*\<\/\w+\>',                          # HTML/XML tags
        r'\bSELECT\b|\bINSERT\b|\bUPDATE\b',            # SQL
        r'\=.*\;',                                      # Variable assignments in many languages
    ]
    
    # Combine patterns into one regex
    combined_pattern = re.compile('|'.join(code_patterns), re.IGNORECASE)
    
    return bool(combined_pattern.search(text))

# Data cleaning

## Tulu Yousef

In [8]:
tulu_yousef = load_dataset('khoubrane-yousef/tulu-v2-sft-mixture-role-split-english-darija', split='train')

In [9]:
tulu_yousef

Dataset({
    features: ['dataset', 'id', 'role', 'english', 'darija'],
    num_rows: 407986
})

In [10]:
# 
dedup_subset_columns = ['english', 'darija']
dedup_tulu_yousef = remove_duplicates(tulu_yousef, dedup_subset_columns)

In [11]:
dedup_tulu_yousef

Dataset({
    features: ['dataset', 'id', 'role', 'english', 'darija', '__index_level_0__'],
    num_rows: 387965
})

In [12]:
# Filter out rows that contain code
filtered_dataset = dedup_tulu_yousef.filter(lambda example: not contains_code(example['darija']))

Filter: 100%|██████████| 387965/387965 [01:37<00:00, 3989.94 examples/s]


In [13]:
filtered_dataset

Dataset({
    features: ['dataset', 'id', 'role', 'english', 'darija', '__index_level_0__'],
    num_rows: 352519
})

In [14]:
filtered_dataset = filtered_dataset.remove_columns(['__index_level_0__'])

In [15]:
filtered_dataset

Dataset({
    features: ['dataset', 'id', 'role', 'english', 'darija'],
    num_rows: 352519
})

In [16]:
# Define the new column order
new_column_order = ['english', 'darija', 'id', 'role', 'dataset']

# Reorder the dataset columns
reordered_data = {col: filtered_dataset[col] for col in new_column_order}
reordered_data =  Dataset.from_dict(reordered_data)

In [17]:
reordered_data

Dataset({
    features: ['english', 'darija', 'id', 'role', 'dataset'],
    num_rows: 352519
})

In [18]:
# reordered_data.push_to_hub("atlasia/tulu-v2-sft-mixture-role-split-english-darija-cleaned", commit_message="Removed duplicates and samples containing code")

## Trans data learnmoroccan

In [19]:
ts_new = load_dataset("BounharAbdelaziz/Translated-dataset-new", split="train").to_pandas()

In [20]:
ts_new.columns

Index(['Unnamed: 0', 'english', 'darija_ar', 'darija_ltn', 'arabic', 'french',
       'spanish', 'chinese', 'russian', 'german', 'japanese', 'italian',
       'korean', 'hindi', 'portuguese', 'greec', 'dataset_source',
       'original_english', 'auto_translated'],
      dtype='object')

In [21]:
ts_new_needed = ts_new[['english', 'darija_ar', 'dataset_source']]

In [22]:
ts_new_needed

,english,darija_ar,dataset_source
0,More than half of the Makhzen's expenditures w...,التقريب أكثر من نص مصاريف المخزن كانت كتمشي لب...,medmac01/moroccan_history_qa
1,What were the consequences of the Makhzen's ex...,ماهي عواقب مصاريف المخزن على الاقتصاد المغربي؟,medmac01/moroccan_history_qa
2,The Makhzen's expenditures led to a deteriorat...,نفقات المخزن أدت إلى تدهور الاقتصاد المغربي، ح...,medmac01/moroccan_history_qa
3,"In the 1890s, the French administration and mi...",في تسعينيات القرن التاسع عشر، طالبت الإدارة وا...,medmac01/moroccan_history_qa
4,What were the main reasons behind the French a...,ما كانو الأسباب الرئيسية اللي دفعو فرنسا باش ت...,medmac01/moroccan_history_qa
...,...,...,...
169801,None,واخا الموقف محبط، خاصنا نتفهمو بلي الصحافة عند...,MBZUAI-Paris/MoroccanSocialMedia-MultiGen
169802,None,مامزيانش نهضرو على الناس بهاد الطريقة. خاصنا ن...,MBZUAI-Paris/MoroccanSocialMedia-MultiGen
169803,None,ماشي فكرة مزيانة هادي أخويا. خاصنا نركزو على ا...,MBZUAI-Paris/MoroccanSocialMedia-MultiGen
169804,None,ما خاصناش نحكمو على الموسيقى ديال الناس الاخري...,MBZUAI-Paris/MoroccanSocialMedia-MultiGen


In [23]:
ts_new_needed = ts_new_needed.dropna()

In [24]:
ts_new_needed

,english,darija_ar,dataset_source
0,More than half of the Makhzen's expenditures w...,التقريب أكثر من نص مصاريف المخزن كانت كتمشي لب...,medmac01/moroccan_history_qa
1,What were the consequences of the Makhzen's ex...,ماهي عواقب مصاريف المخزن على الاقتصاد المغربي؟,medmac01/moroccan_history_qa
2,The Makhzen's expenditures led to a deteriorat...,نفقات المخزن أدت إلى تدهور الاقتصاد المغربي، ح...,medmac01/moroccan_history_qa
3,"In the 1890s, the French administration and mi...",في تسعينيات القرن التاسع عشر، طالبت الإدارة وا...,medmac01/moroccan_history_qa
4,What were the main reasons behind the French a...,ما كانو الأسباب الرئيسية اللي دفعو فرنسا باش ت...,medmac01/moroccan_history_qa
...,...,...,...
42314,You can arrange your travel by contacting CHAF...,تقدر ترتب رحلتك من خلال الاتصال بالشافعي فوياج...,aboutaleb/moroccantourism
42315,"In Marrakech, CHAFAI VOYAGES is a reliable tra...",في مراكش، شافعي فواياج هي وكالة سفر موثوق فيها...,aboutaleb/moroccantourism
42316,"Yes, CHAFAI VOYAGES in Marrakech offers guided...",نعم، شافعي فواياج في مراكش كيقدمو جولات سياحية...,aboutaleb/moroccantourism
42317,"In Marrakech, you have several options, includ...",في مراكش، عندك بزاف د الختيارات، من بينهم شافع...,aboutaleb/moroccantourism


In [25]:
ts_new_needed_dedup = ts_new_needed.drop_duplicates(['english', 'darija_ar'])

In [26]:
ts_new_needed_dedup

,english,darija_ar,dataset_source
0,More than half of the Makhzen's expenditures w...,التقريب أكثر من نص مصاريف المخزن كانت كتمشي لب...,medmac01/moroccan_history_qa
1,What were the consequences of the Makhzen's ex...,ماهي عواقب مصاريف المخزن على الاقتصاد المغربي؟,medmac01/moroccan_history_qa
2,The Makhzen's expenditures led to a deteriorat...,نفقات المخزن أدت إلى تدهور الاقتصاد المغربي، ح...,medmac01/moroccan_history_qa
3,"In the 1890s, the French administration and mi...",في تسعينيات القرن التاسع عشر، طالبت الإدارة وا...,medmac01/moroccan_history_qa
4,What were the main reasons behind the French a...,ما كانو الأسباب الرئيسية اللي دفعو فرنسا باش ت...,medmac01/moroccan_history_qa
...,...,...,...
42314,You can arrange your travel by contacting CHAF...,تقدر ترتب رحلتك من خلال الاتصال بالشافعي فوياج...,aboutaleb/moroccantourism
42315,"In Marrakech, CHAFAI VOYAGES is a reliable tra...",في مراكش، شافعي فواياج هي وكالة سفر موثوق فيها...,aboutaleb/moroccantourism
42316,"Yes, CHAFAI VOYAGES in Marrakech offers guided...",نعم، شافعي فواياج في مراكش كيقدمو جولات سياحية...,aboutaleb/moroccantourism
42317,"In Marrakech, you have several options, includ...",في مراكش، عندك بزاف د الختيارات، من بينهم شافع...,aboutaleb/moroccantourism


In [27]:
ts_new_final = Dataset.from_pandas(ts_new_needed_dedup)

In [28]:
ts_new_final

Dataset({
    features: ['english', 'darija_ar', 'dataset_source', '__index_level_0__'],
    num_rows: 42027
})

In [29]:
# ts_new_final.push_to_hub("BounharAbdelaziz/Translated-dataset-learnmoroccan-42K")

In [30]:
ds = load_dataset("BounharAbdelaziz/darija-translation-v6", split="train").to_pandas()

In [31]:
ds = ds[['english', 'darija_ar', 'dataset_name']]

In [32]:
ds

,english,darija_ar,dataset_name
0,Machine Learning is a fast moving domain,الظومين ديال الماشين لورنين كاي تطور دغيا,None
1,More than half of the Makhzen's expenditures w...,التقريب أكثر من نص مصاريف المخزن كانت كتمشي لب...,medmac01/moroccan_history_qa
2,What were the consequences of the Makhzen's ex...,ماهي عواقب مصاريف المخزن على الاقتصاد المغربي؟,medmac01/moroccan_history_qa
3,The Makhzen's expenditures led to a deteriorat...,نفقات المخزن أدت إلى تدهور الاقتصاد المغربي، ح...,medmac01/moroccan_history_qa
4,"In the 1890s, the French administration and mi...",في تسعينيات القرن التاسع عشر، طالبت الإدارة وا...,medmac01/moroccan_history_qa
...,...,...,...
228179,"I apologize, but I do not feel comfortable tra...",...عينيييها محلووولين على آاااخرهم و قاااع رجل...,atlasia/darija_english/stories
228180,The mother returned from the road before reach...,...نهار جا رجعاتو ماما من الطريق قبل ما يوصل ل...,atlasia/darija_english/stories
228181,You want to escape your problems by getting ma...,...لي بغات تهنى من مشاكلها تتزوج...بحال الى زو...,atlasia/darija_english/stories
228182,He felt relatively relieved from the anger tha...,... حس بللي رتااح نسبيااا من دااك الغضب لي اجت...,atlasia/darija_english/stories


In [33]:
ds = ds.dropna()

In [34]:
ds

,english,darija_ar,dataset_name
1,More than half of the Makhzen's expenditures w...,التقريب أكثر من نص مصاريف المخزن كانت كتمشي لب...,medmac01/moroccan_history_qa
2,What were the consequences of the Makhzen's ex...,ماهي عواقب مصاريف المخزن على الاقتصاد المغربي؟,medmac01/moroccan_history_qa
3,The Makhzen's expenditures led to a deteriorat...,نفقات المخزن أدت إلى تدهور الاقتصاد المغربي، ح...,medmac01/moroccan_history_qa
4,"In the 1890s, the French administration and mi...",في تسعينيات القرن التاسع عشر، طالبت الإدارة وا...,medmac01/moroccan_history_qa
5,What were the main reasons behind the French a...,ما كانو الأسباب الرئيسية اللي دفعو فرنسا باش ت...,medmac01/moroccan_history_qa
...,...,...,...
228179,"I apologize, but I do not feel comfortable tra...",...عينيييها محلووولين على آاااخرهم و قاااع رجل...,atlasia/darija_english/stories
228180,The mother returned from the road before reach...,...نهار جا رجعاتو ماما من الطريق قبل ما يوصل ل...,atlasia/darija_english/stories
228181,You want to escape your problems by getting ma...,...لي بغات تهنى من مشاكلها تتزوج...بحال الى زو...,atlasia/darija_english/stories
228182,He felt relatively relieved from the anger tha...,... حس بللي رتااح نسبيااا من دااك الغضب لي اجت...,atlasia/darija_english/stories


In [35]:
import pandas as pd
df = pd.concat([ds, ts_new_needed_dedup], ignore_index=True)

In [36]:
df

,english,darija_ar,dataset_name,dataset_source
0,More than half of the Makhzen's expenditures w...,التقريب أكثر من نص مصاريف المخزن كانت كتمشي لب...,medmac01/moroccan_history_qa,NaN
1,What were the consequences of the Makhzen's ex...,ماهي عواقب مصاريف المخزن على الاقتصاد المغربي؟,medmac01/moroccan_history_qa,NaN
2,The Makhzen's expenditures led to a deteriorat...,نفقات المخزن أدت إلى تدهور الاقتصاد المغربي، ح...,medmac01/moroccan_history_qa,NaN
3,"In the 1890s, the French administration and mi...",في تسعينيات القرن التاسع عشر، طالبت الإدارة وا...,medmac01/moroccan_history_qa,NaN
4,What were the main reasons behind the French a...,ما كانو الأسباب الرئيسية اللي دفعو فرنسا باش ت...,medmac01/moroccan_history_qa,NaN
...,...,...,...,...
270185,You can arrange your travel by contacting CHAF...,تقدر ترتب رحلتك من خلال الاتصال بالشافعي فوياج...,NaN,aboutaleb/moroccantourism
270186,"In Marrakech, CHAFAI VOYAGES is a reliable tra...",في مراكش، شافعي فواياج هي وكالة سفر موثوق فيها...,NaN,aboutaleb/moroccantourism
270187,"Yes, CHAFAI VOYAGES in Marrakech offers guided...",نعم، شافعي فواياج في مراكش كيقدمو جولات سياحية...,NaN,aboutaleb/moroccantourism
270188,"In Marrakech, you have several options, includ...",في مراكش، عندك بزاف د الختيارات، من بينهم شافع...,NaN,aboutaleb/moroccantourism


In [37]:
df_dedup = df.drop_duplicates(['english', 'darija_ar'])

In [38]:
df_dedup

,english,darija_ar,dataset_name,dataset_source
0,More than half of the Makhzen's expenditures w...,التقريب أكثر من نص مصاريف المخزن كانت كتمشي لب...,medmac01/moroccan_history_qa,NaN
1,What were the consequences of the Makhzen's ex...,ماهي عواقب مصاريف المخزن على الاقتصاد المغربي؟,medmac01/moroccan_history_qa,NaN
2,The Makhzen's expenditures led to a deteriorat...,نفقات المخزن أدت إلى تدهور الاقتصاد المغربي، ح...,medmac01/moroccan_history_qa,NaN
3,"In the 1890s, the French administration and mi...",في تسعينيات القرن التاسع عشر، طالبت الإدارة وا...,medmac01/moroccan_history_qa,NaN
4,What were the main reasons behind the French a...,ما كانو الأسباب الرئيسية اللي دفعو فرنسا باش ت...,medmac01/moroccan_history_qa,NaN
...,...,...,...,...
270185,You can arrange your travel by contacting CHAF...,تقدر ترتب رحلتك من خلال الاتصال بالشافعي فوياج...,NaN,aboutaleb/moroccantourism
270186,"In Marrakech, CHAFAI VOYAGES is a reliable tra...",في مراكش، شافعي فواياج هي وكالة سفر موثوق فيها...,NaN,aboutaleb/moroccantourism
270187,"Yes, CHAFAI VOYAGES in Marrakech offers guided...",نعم، شافعي فواياج في مراكش كيقدمو جولات سياحية...,NaN,aboutaleb/moroccantourism
270188,"In Marrakech, you have several options, includ...",في مراكش، عندك بزاف د الختيارات، من بينهم شافع...,NaN,aboutaleb/moroccantourism


In [39]:
# Fill NaN values in 'dataset_name' with corresponding non-NaN values from 'dataset_source' or any other relevant column
df_dedup['dataset_source'] = df['dataset_source'].fillna(df['dataset_name'])

/tmp/ipykernel_1604732/331978093.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dedup['dataset_source'] = df['dataset_source'].fillna(df['dataset_name'])


In [40]:
df_dedup['dataset_source'].value_counts()

dataset_source
NQ-Open-AYA                                   143740
aboutaleb/moroccantourism                      30445
common-crawl                                   19660
doda                                           13985
medmac01/moroccan_history_qa                   11179
atlasia/darija_english/stories                  3000
learnmoroccan.com                               1763
loecsen.com                                     1704
SoftAge-AI/sft-conversational_dataset            794
auto-math                                        415
stanford                                         352
bourbouh/moroccan-darija-youtube-subtitles       248
translation-rows                                 246
wikihow                                          140
Name: count, dtype: int64

In [41]:
df_dedup = df_dedup[['english', 'darija_ar', 'dataset_source']]

In [42]:
df_dedup

,english,darija_ar,dataset_source
0,More than half of the Makhzen's expenditures w...,التقريب أكثر من نص مصاريف المخزن كانت كتمشي لب...,medmac01/moroccan_history_qa
1,What were the consequences of the Makhzen's ex...,ماهي عواقب مصاريف المخزن على الاقتصاد المغربي؟,medmac01/moroccan_history_qa
2,The Makhzen's expenditures led to a deteriorat...,نفقات المخزن أدت إلى تدهور الاقتصاد المغربي، ح...,medmac01/moroccan_history_qa
3,"In the 1890s, the French administration and mi...",في تسعينيات القرن التاسع عشر، طالبت الإدارة وا...,medmac01/moroccan_history_qa
4,What were the main reasons behind the French a...,ما كانو الأسباب الرئيسية اللي دفعو فرنسا باش ت...,medmac01/moroccan_history_qa
...,...,...,...
270185,You can arrange your travel by contacting CHAF...,تقدر ترتب رحلتك من خلال الاتصال بالشافعي فوياج...,aboutaleb/moroccantourism
270186,"In Marrakech, CHAFAI VOYAGES is a reliable tra...",في مراكش، شافعي فواياج هي وكالة سفر موثوق فيها...,aboutaleb/moroccantourism
270187,"Yes, CHAFAI VOYAGES in Marrakech offers guided...",نعم، شافعي فواياج في مراكش كيقدمو جولات سياحية...,aboutaleb/moroccantourism
270188,"In Marrakech, you have several options, includ...",في مراكش، عندك بزاف د الختيارات، من بينهم شافع...,aboutaleb/moroccantourism


In [43]:
df_tulu = reordered_data.to_pandas()

In [44]:
df_tulu

,english,darija,id,role,dataset
0,"Q: A.S. Roma, chairman, James Pallotta; James ...",س: أ.س. روما، الرئيس، جيمس بالوتا؛ جيمس بالوتا...,flan_v2_1,user,flan_v2
1,English is one of the languages spoken in Sing...,الإنجليزية هي وحدة من اللغات اللي كيتكلمو بيها...,flan_v2_1,assistant,flan_v2
2,Write a movie review.\nconfidently \nWrite a m...,كتب ريفيو ديال فيلم.\nبكل ثقة \nكتب ريفيو ديال...,flan_v2_2,user,flan_v2
3,", should n't the reality seem at least passabl...",، واش ماخاصش الواقع يبان على الأقل شوية حقيقي؟,flan_v2_2,assistant,flan_v2
4,"Problem: Data: name = Midsummer House, eatType...",مشكل: داتا: سميتو = Midsummer House، نوع الماك...,flan_v2_3,user,flan_v2
...,...,...,...,...,...
352514,"To ensure that each passenger has a ride, and ...",باش نضمنو أن كل راكب عندو رحلة، وما يبقى حتى ب...,wizardlm_gIdmqym,assistant,wizardlm
352515,Rewrite the following sentence to make it more...,عاود كتابة الجملة التالية باش تكون أقصر وزيد ط...,wizardlm_alpaca_11298,user,wizardlm
352516,"""Generate a SQL query grouping the number of n...","""جيب استعلام SQL كيجمع عدد الدراري اللي كيديرو...",wizardlm_alpaca_11298,assistant,wizardlm
352517,How can someone ensure their data is secure in...,كيفاش واحد يقدر يضمن أن البيانات ديالو آمنة فب...,wizardlm_alpaca_18585,user,wizardlm


In [45]:
df_tulu.columns = ['english', 'darija_ar', 'id', 'role', 'dataset_source']

In [46]:
df_tulu = df_tulu[['english', 'darija_ar', 'dataset_source', 'id', 'role']]

In [47]:
df_tulu

,english,darija_ar,dataset_source,id,role
0,"Q: A.S. Roma, chairman, James Pallotta; James ...",س: أ.س. روما، الرئيس، جيمس بالوتا؛ جيمس بالوتا...,flan_v2,flan_v2_1,user
1,English is one of the languages spoken in Sing...,الإنجليزية هي وحدة من اللغات اللي كيتكلمو بيها...,flan_v2,flan_v2_1,assistant
2,Write a movie review.\nconfidently \nWrite a m...,كتب ريفيو ديال فيلم.\nبكل ثقة \nكتب ريفيو ديال...,flan_v2,flan_v2_2,user
3,", should n't the reality seem at least passabl...",، واش ماخاصش الواقع يبان على الأقل شوية حقيقي؟,flan_v2,flan_v2_2,assistant
4,"Problem: Data: name = Midsummer House, eatType...",مشكل: داتا: سميتو = Midsummer House، نوع الماك...,flan_v2,flan_v2_3,user
...,...,...,...,...,...
352514,"To ensure that each passenger has a ride, and ...",باش نضمنو أن كل راكب عندو رحلة، وما يبقى حتى ب...,wizardlm,wizardlm_gIdmqym,assistant
352515,Rewrite the following sentence to make it more...,عاود كتابة الجملة التالية باش تكون أقصر وزيد ط...,wizardlm,wizardlm_alpaca_11298,user
352516,"""Generate a SQL query grouping the number of n...","""جيب استعلام SQL كيجمع عدد الدراري اللي كيديرو...",wizardlm,wizardlm_alpaca_11298,assistant
352517,How can someone ensure their data is secure in...,كيفاش واحد يقدر يضمن أن البيانات ديالو آمنة فب...,wizardlm,wizardlm_alpaca_18585,user


In [48]:
df_dedup

,english,darija_ar,dataset_source
0,More than half of the Makhzen's expenditures w...,التقريب أكثر من نص مصاريف المخزن كانت كتمشي لب...,medmac01/moroccan_history_qa
1,What were the consequences of the Makhzen's ex...,ماهي عواقب مصاريف المخزن على الاقتصاد المغربي؟,medmac01/moroccan_history_qa
2,The Makhzen's expenditures led to a deteriorat...,نفقات المخزن أدت إلى تدهور الاقتصاد المغربي، ح...,medmac01/moroccan_history_qa
3,"In the 1890s, the French administration and mi...",في تسعينيات القرن التاسع عشر، طالبت الإدارة وا...,medmac01/moroccan_history_qa
4,What were the main reasons behind the French a...,ما كانو الأسباب الرئيسية اللي دفعو فرنسا باش ت...,medmac01/moroccan_history_qa
...,...,...,...
270185,You can arrange your travel by contacting CHAF...,تقدر ترتب رحلتك من خلال الاتصال بالشافعي فوياج...,aboutaleb/moroccantourism
270186,"In Marrakech, CHAFAI VOYAGES is a reliable tra...",في مراكش، شافعي فواياج هي وكالة سفر موثوق فيها...,aboutaleb/moroccantourism
270187,"Yes, CHAFAI VOYAGES in Marrakech offers guided...",نعم، شافعي فواياج في مراكش كيقدمو جولات سياحية...,aboutaleb/moroccantourism
270188,"In Marrakech, you have several options, includ...",في مراكش، عندك بزاف د الختيارات، من بينهم شافع...,aboutaleb/moroccantourism


In [49]:
df_dedup['id'] = None
df_dedup['role'] = None

In [50]:
final_translation_df = pd.concat([df_dedup, df_tulu])

In [51]:
final_translation_df

,english,darija_ar,dataset_source,id,role
0,More than half of the Makhzen's expenditures w...,التقريب أكثر من نص مصاريف المخزن كانت كتمشي لب...,medmac01/moroccan_history_qa,None,None
1,What were the consequences of the Makhzen's ex...,ماهي عواقب مصاريف المخزن على الاقتصاد المغربي؟,medmac01/moroccan_history_qa,None,None
2,The Makhzen's expenditures led to a deteriorat...,نفقات المخزن أدت إلى تدهور الاقتصاد المغربي، ح...,medmac01/moroccan_history_qa,None,None
3,"In the 1890s, the French administration and mi...",في تسعينيات القرن التاسع عشر، طالبت الإدارة وا...,medmac01/moroccan_history_qa,None,None
4,What were the main reasons behind the French a...,ما كانو الأسباب الرئيسية اللي دفعو فرنسا باش ت...,medmac01/moroccan_history_qa,None,None
...,...,...,...,...,...
352514,"To ensure that each passenger has a ride, and ...",باش نضمنو أن كل راكب عندو رحلة، وما يبقى حتى ب...,wizardlm,wizardlm_gIdmqym,assistant
352515,Rewrite the following sentence to make it more...,عاود كتابة الجملة التالية باش تكون أقصر وزيد ط...,wizardlm,wizardlm_alpaca_11298,user
352516,"""Generate a SQL query grouping the number of n...","""جيب استعلام SQL كيجمع عدد الدراري اللي كيديرو...",wizardlm,wizardlm_alpaca_11298,assistant
352517,How can someone ensure their data is secure in...,كيفاش واحد يقدر يضمن أن البيانات ديالو آمنة فب...,wizardlm,wizardlm_alpaca_18585,user


In [52]:
# Replace '<unk>' with '-'
final_translation_df['darija_ar'] = final_translation_df['darija_ar'].str.replace(' <unk>', '-', regex=False)
final_translation_df['english'] = final_translation_df['english'].str.replace(' <unk>', '-', regex=False)

final_translation_df['darija_ar'] = final_translation_df['darija_ar'].str.replace('<unk>', '', regex=False)
final_translation_df['english'] = final_translation_df['english'].str.replace('<unk>', '', regex=False)

In [53]:
final_translation_df

,english,darija_ar,dataset_source,id,role
0,More than half of the Makhzen's expenditures w...,التقريب أكثر من نص مصاريف المخزن كانت كتمشي لب...,medmac01/moroccan_history_qa,None,None
1,What were the consequences of the Makhzen's ex...,ماهي عواقب مصاريف المخزن على الاقتصاد المغربي؟,medmac01/moroccan_history_qa,None,None
2,The Makhzen's expenditures led to a deteriorat...,نفقات المخزن أدت إلى تدهور الاقتصاد المغربي، ح...,medmac01/moroccan_history_qa,None,None
3,"In the 1890s, the French administration and mi...",في تسعينيات القرن التاسع عشر، طالبت الإدارة وا...,medmac01/moroccan_history_qa,None,None
4,What were the main reasons behind the French a...,ما كانو الأسباب الرئيسية اللي دفعو فرنسا باش ت...,medmac01/moroccan_history_qa,None,None
...,...,...,...,...,...
352514,"To ensure that each passenger has a ride, and ...",باش نضمنو أن كل راكب عندو رحلة، وما يبقى حتى ب...,wizardlm,wizardlm_gIdmqym,assistant
352515,Rewrite the following sentence to make it more...,عاود كتابة الجملة التالية باش تكون أقصر وزيد ط...,wizardlm,wizardlm_alpaca_11298,user
352516,"""Generate a SQL query grouping the number of n...","""جيب استعلام SQL كيجمع عدد الدراري اللي كيديرو...",wizardlm,wizardlm_alpaca_11298,assistant
352517,How can someone ensure their data is secure in...,كيفاش واحد يقدر يضمن أن البيانات ديالو آمنة فب...,wizardlm,wizardlm_alpaca_18585,user


In [54]:
final_translation_ds = Dataset.from_pandas(final_translation_df)

In [56]:
final_translation_ds

Dataset({
    features: ['english', 'darija_ar', 'dataset_source', 'id', 'role', '__index_level_0__'],
    num_rows: 580190
})

In [57]:
final_translation_ds = final_translation_ds.remove_columns(['__index_level_0__'])

In [58]:
final_translation_ds

Dataset({
    features: ['english', 'darija_ar', 'dataset_source', 'id', 'role'],
    num_rows: 580190
})

In [60]:
final_translation_ds.push_to_hub("BounharAbdelaziz/Terjman-v2-English-Darija-Dataset-580K")

Uploading the dataset shards: 100%|██████████| 2/2 [00:14<00:00,  7.25s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/BounharAbdelaziz/Terjman-v2-English-Darija-Dataset-580K/commit/131c31190d675c0a3b3c690e7b2b4037ded5a4ed', commit_message='Upload dataset', commit_description='', oid='131c31190d675c0a3b3c690e7b2b4037ded5a4ed', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/BounharAbdelaziz/Terjman-v2-English-Darija-Dataset-580K', endpoint='https://huggingface.co', repo_type='dataset', repo_id='BounharAbdelaziz/Terjman-v2-English-Darija-Dataset-580K'), pr_revision=None, pr_num=None)

## ASR data

In [61]:
asr = load_dataset("BounharAbdelaziz/Morocco-Darija-ASR", split='train')

Generating validation split: 100%|██████████| 300/300 [00:00<00:00, 356.28 examples/s]


In [64]:
asr

Dataset({
    features: ['audio', 'transcription', 'language', 'dataset_source'],
    num_rows: 9401
})

In [65]:
asr.select_columns(["transcription"])

Dataset({
    features: ['transcription'],
    num_rows: 9401
})

In [99]:
df_asr = asr.select_columns(["transcription"]).to_pandas()

In [100]:
df_asr.columns = ['text']

In [101]:
df_asr

,text
0,ونقدرو نخليو تقريبا نفس القياس بشرط ديك الساعة...
1,هو ان يعني اه يعني ان الأستاذ ايلا ما كانش كيح...
2,غادي يخرجو واحد القانون صفر واحد صفر صفر تنظيم...
3,نشوفو واحد القضية نقطة الضعف ديالي هي الوالدة ...
4,شنو المشاكل لي كيبانو ليك فالمغرب فهاد الجامعا...
...,...
9396,اه ما شاء الله عليه هاكا وهادشي كنظن خلاني تا ...
9397,شكونا هما هاد الناس؟ شافني شي واحد من داك النا...
9398,ديها شوية فصحتك الفلوس راه كا تمشي و تجي المهم...
9399,مشروع عن قيم عن مبادئ للأسف اليوم ملي فتحنا جر...


In [102]:
df_trans = final_translation_df[['darija_ar']]

In [103]:
df_trans

,darija_ar
0,التقريب أكثر من نص مصاريف المخزن كانت كتمشي لب...
1,ماهي عواقب مصاريف المخزن على الاقتصاد المغربي؟
2,نفقات المخزن أدت إلى تدهور الاقتصاد المغربي، ح...
3,في تسعينيات القرن التاسع عشر، طالبت الإدارة وا...
4,ما كانو الأسباب الرئيسية اللي دفعو فرنسا باش ت...
...,...
352514,باش نضمنو أن كل راكب عندو رحلة، وما يبقى حتى ب...
352515,عاود كتابة الجملة التالية باش تكون أقصر وزيد ط...
352516,"""جيب استعلام SQL كيجمع عدد الدراري اللي كيديرو..."
352517,كيفاش واحد يقدر يضمن أن البيانات ديالو آمنة فب...


In [104]:
df_trans.columns = ['text']

In [122]:
asr_2 = load_dataset('BounharAbdelaziz/Youtube-Commons-Moroccan-Darija-14h47', split='train')

In [123]:
df_asr_2 = asr_2.select_columns(["transcription"]).to_pandas()

In [125]:
df_asr_2.columns = ['text']

In [126]:
concat_df = pd.concat([df_trans, df_asr, df_asr_2])
concat_df['dataset_source'] = "atlasia/Terjman-v2-English-Darija-Dataset"
concat_df['metadata'] = str({})

In [127]:
concat_df

,text,dataset_source,metadata
0,التقريب أكثر من نص مصاريف المخزن كانت كتمشي لب...,atlasia/Terjman-v2-English-Darija-Dataset,{}
1,ماهي عواقب مصاريف المخزن على الاقتصاد المغربي؟,atlasia/Terjman-v2-English-Darija-Dataset,{}
2,نفقات المخزن أدت إلى تدهور الاقتصاد المغربي، ح...,atlasia/Terjman-v2-English-Darija-Dataset,{}
3,في تسعينيات القرن التاسع عشر، طالبت الإدارة وا...,atlasia/Terjman-v2-English-Darija-Dataset,{}
4,ما كانو الأسباب الرئيسية اللي دفعو فرنسا باش ت...,atlasia/Terjman-v2-English-Darija-Dataset,{}
...,...,...,...
5207,لعبة كاتوقف ليك، خصك تقلب كيفاش تفانتاك، كيفاش...,atlasia/Terjman-v2-English-Darija-Dataset,{}
5208,واحد الحاجه اللي كتلقى راسك نتا بحال for examp...,atlasia/Terjman-v2-English-Darija-Dataset,{}
5209,ولكن i think i think i think we should start f...,atlasia/Terjman-v2-English-Darija-Dataset,{}
5210,identifying the term نعرفوا الترمز في المغرب ف...,atlasia/Terjman-v2-English-Darija-Dataset,{}


In [128]:
new_data = DatasetDict({
    "train": Dataset.from_pandas(concat_df)
})

In [129]:
new_data

DatasetDict({
    train: Dataset({
        features: ['text', 'dataset_source', 'metadata', '__index_level_0__'],
        num_rows: 594803
    })
})

# Adding to Al Atlas

In [73]:
al_atlas_ds = load_dataset("atlasia/AL-Atlas-Moroccan-Darija-Pretraining-Dataset")

Generating test split: 100%|██████████| 1875/1875 [00:00<00:00, 428853.75 examples/s]


In [74]:
al_atlas_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'dataset_source', 'metadata'],
        num_rows: 2818353
    })
    test: Dataset({
        features: ['text', 'dataset_source', 'metadata'],
        num_rows: 1875
    })
})

In [110]:
new_data

DatasetDict({
    train: Dataset({
        features: ['text', 'dataset_source', 'metadata', '__index_level_0__'],
        num_rows: 589591
    })
})

In [130]:
from datasets import concatenate_datasets

al_atlas_ds_train = concatenate_datasets([al_atlas_ds['train'], new_data['train']])

In [131]:
al_atlas_ds_train

Dataset({
    features: ['text', 'dataset_source', 'metadata', '__index_level_0__'],
    num_rows: 3413156
})

In [132]:
al_atlas_ds_train = al_atlas_ds_train.remove_columns(['__index_level_0__'])

In [133]:
new_al_atlas_ds = DatasetDict({
    "train": al_atlas_ds_train,
    "test": al_atlas_ds['test']
})

In [134]:
new_al_atlas_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'dataset_source', 'metadata'],
        num_rows: 3413156
    })
    test: Dataset({
        features: ['text', 'dataset_source', 'metadata'],
        num_rows: 1875
    })
})

In [135]:
new_al_atlas_ds.push_to_hub('atlasia/AL-Atlas-Moroccan-Darija-Pretraining-Dataset', commit_description="Added translation and ASR datas")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  3.51it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/atlasia/AL-Atlas-Moroccan-Darija-Pretraining-Dataset/commit/6b0b60a377275f37437be4110f743e07cb23b067', commit_message='Upload dataset', commit_description='Added translation and ASR datas', oid='6b0b60a377275f37437be4110f743e07cb23b067', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/atlasia/AL-Atlas-Moroccan-Darija-Pretraining-Dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='atlasia/AL-Atlas-Moroccan-Darija-Pretraining-Dataset'), pr_revision=None, pr_num=None)